In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio

import pandas as pd
import numpy as np
import librosa
import glob
import os

import csv
import io
from tqdm.auto import tqdm

from IPython.display import Audio

2024-05-21 19:44:30.080226: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 19:44:30.168454: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


ModuleNotFoundError: No module named 'tensorflow_hub'

In [2]:
model = hub.load('https://kaggle.com/models/google/bird-vocalization-classifier/frameworks/tensorFlow2/variations/bird-vocalization-classifier/versions/4')
labels_path = hub.resolve('https://kaggle.com/models/google/bird-vocalization-classifier/frameworks/tensorFlow2/variations/bird-vocalization-classifier/versions/4') + "/assets/label.csv"

Attaching model 'google/bird-vocalization-classifier/tensorflow2/bird-vocalization-classifier/4' to your Kaggle notebook...
Attaching model 'google/bird-vocalization-classifier/tensorflow2/bird-vocalization-classifier/4' to your Kaggle notebook...


In [3]:
# Find the name of the class with the top score when mean-aggregated across frames.
def class_names_from_csv(class_map_csv_text):
    """Returns list of class names corresponding to score vector."""
    with open(labels_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        class_names = [mid[0] for mid in csv_reader]
        return class_names[1:]

## note that the bird classifier classifies a much larger set of birds than the
## competition, so we need to load the model's set of class names or else our 
## indices will be off.
classes = class_names_from_csv(labels_path)

In [4]:
# Use on except pretraining dataset
train_metadata = pd.read_csv("/kaggle/input/birdclef-2024/train_metadata.csv")

# Use on pretraining dataset
#train_metadata = pd.read_csv("/kaggle/input/birdclef-2023-additional/pretrain_metadata_10fold.csv")


competition_classes = sorted(train_metadata.primary_label.unique())

forced_defaults = 0
competition_class_map = []
for c in competition_classes:
    try:
        i = classes.index(c)
        competition_class_map.append(i)
    except:
        print(c)
        competition_class_map.append(0)
        forced_defaults += 1
        
## this is the count of classes not supported by our pretrained model
## you could choose to simply not predict these, set a default as above,
## or create your own model using the pretrained model as a base.
forced_defaults

bkrfla1
indrol2


2

In [5]:
train_metadata

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename
0,asbfly,[],['call'],39.2297,118.1987,Muscicapa dauurica,Asian Brown Flycatcher,Matt Slaymaker,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/134896,asbfly/XC134896.ogg
1,asbfly,[],['song'],51.4030,104.6401,Muscicapa dauurica,Asian Brown Flycatcher,Magnus Hellström,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/164848,asbfly/XC164848.ogg
2,asbfly,[],['song'],36.3319,127.3555,Muscicapa dauurica,Asian Brown Flycatcher,Stuart Fisher,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/175797,asbfly/XC175797.ogg
3,asbfly,[],['call'],21.1697,70.6005,Muscicapa dauurica,Asian Brown Flycatcher,vir joshi,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/207738,asbfly/XC207738.ogg
4,asbfly,[],['call'],15.5442,73.7733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209218,asbfly/XC209218.ogg
...,...,...,...,...,...,...,...,...,...,...,...,...
24454,zitcis1,[],[''],43.5925,4.5434,Cisticola juncidis,Zitting Cisticola,Chèvremont Fabian,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://xeno-canto.org/845747,zitcis1/XC845747.ogg
24455,zitcis1,[],[''],43.5925,4.5434,Cisticola juncidis,Zitting Cisticola,Chèvremont Fabian,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/845817,zitcis1/XC845817.ogg
24456,zitcis1,[],[''],51.1207,4.5607,Cisticola juncidis,Zitting Cisticola,Wim Jacobs,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/856176,zitcis1/XC856176.ogg
24457,zitcis1,[],[''],41.5607,-8.4236,Cisticola juncidis,Zitting Cisticola,Jorge Leitão,Creative Commons Attribution-NonCommercial-Sha...,4.5,https://xeno-canto.org/856723,zitcis1/XC856723.ogg


In [6]:
def frame_audio(
      audio_array: np.ndarray,
      window_size_s: float = 5.0,
      hop_size_s: float = 5.0,
      sample_rate = 32000,
      ) -> np.ndarray:
    
    """Helper function for framing audio for inference."""
    """ using tf.signal """
    if window_size_s is None or window_size_s < 0:
        return audio_array[np.newaxis, :]
    frame_length = int(window_size_s * sample_rate)
    hop_length = int(hop_size_s * sample_rate)
    framed_audio = tf.signal.frame(audio_array, frame_length, hop_length, pad_end=True)
    return framed_audio


def ensure_sample_rate(waveform, original_sample_rate,
                       desired_sample_rate=32000):
    """Resample waveform if required."""
    if original_sample_rate != desired_sample_rate:
        waveform = tfio.audio.resample(waveform, original_sample_rate, desired_sample_rate)
    return desired_sample_rate, waveform

In [7]:
def predict_for_sample(filename, sample_submission, index, frame_limit_secs=None):
    
    # Use on no-call dataset
    #filename = f'/kaggle/input/birdclef2021-background-noise/ff1010bird_nocall/nocall/{filename}'
    
    # Use on additional xeno-canto(CC-BY-NC-SA)
    #dirname = os.path.dirname(filename).split('/')[-1]
    #basename = os.path.basename(filename)
    #filename = f'/kaggle/input/birdclef-2023-additional/xeno-canto/{dirname}/{basename}'
    
    # Use on additional xeno-canto(and CC-BY-NC-ND)
    #dirname = os.path.dirname(filename).split('/')[-1]
    #basename = os.path.basename(filename)
    #filename = f'/kaggle/input/birdclef-2023-additional/xeno-canto_nd/xeno-canto_nd/{dirname}/{basename}'

    # Use on pretraining dataset
    #filename = filename.replace('../', '/kaggle/')
    
    audio, sample_rate = librosa.load(filename, sr=32000, mono=True)
    sample_rate, wav_data = ensure_sample_rate(audio, sample_rate)
    
    fixed_tm = frame_audio(wav_data)
    alllogits, filenames = [], []
#     print(fixed_tm.shape)
    
    frame = 0
#     all_logits, all_embeddings = model.infer_tf(fixed_tm[:1])
#     filenames.append(f"{filename.split('/')[-1][:-4]}_{frame}")
#     alllogits.append(all_logits)
    for window in fixed_tm:
        
        logits, embeddings = model.infer_tf(window[np.newaxis, :])
#         all_logits = np.concatenate([all_logits, logits], axis=0)
        frame += 5
        
        filenames.append(f"{filename.split('/')[-1][:-4]}_{frame}")
        alllogits.append(logits)
    return alllogits, filenames
#     all_logits = np.mean(all_logits, axis=0)  # Averaged over all 5-sec clips
#     sample_submission.loc[index, competition_classes] = all_logits[competition_class_map]
    
    

In [8]:
# Use on training dataset(2023)
# data = pd.read_csv('/kaggle/input/birdclef-2024/train_metadata.csv')

# Use on no-call dataset
#data = pd.read_csv('/kaggle/input/birdclef-2023-additional/ff1010bird_metadata_v1.csv')

# Use on additional xeno-canto(CC-BY-NC-SA)
#data = pd.read_csv('/kaggle/input/birdclef-2023-additional/xeno-canto_audio_meta.csv')

# Use on additional xeno-canto(CC-BY-NC-ND)
#data = pd.read_csv('/kaggle/input/birdclef-2023-additional/xeno-canto_nd_audio_meta.csv')


# Use on training dataset(2023)
# test_samples = list('/kaggle/input/birdclef-2024/unlabeled_soundscapes/' + data.filename)
import glob
test_samples = list(glob.glob(f'/kaggle/input/birdclef-2024/test_soundscapes/*.ogg'))
test_samples = list(glob.glob(f'/kaggle/input/birdclef-2024/train_audio/*/*.ogg'))


if len(test_samples) == 0:
    test_samples = ["/kaggle/input/birdclef-2024/train_audio/asbfly/XC134896.ogg", 
                   "/kaggle/input/birdclef-2024/unlabeled_soundscapes/1000389428.ogg",
                   "/kaggle/input/birdclef-2024/unlabeled_soundscapes/1000424265.ogg"]
# Use on pretraining dataset
#test_samples = list(train_metadata.filename)

# Use on no-call dataset, additional xeno-canto(CC-BY-NC-SA), xeno-canto(CC-BY-NC-ND)
#test_samples = list(data.filename)

len(test_samples)

24459

In [9]:
# Use on training dataset(2023), no-call dataset, xeno-canto(CC-BY-NC-SA), additional xeno-canto(CC-BY-NC-ND)
# sub = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")

# Use on training dataset(2023), pretraining dataset
sample_sub = pd.DataFrame({"filename": test_samples})

# Use on no-call dataset
#sample_sub = pd.read_csv('/kaggle/input/birdclef-2023-additional/ff1010bird_metadata_v1.csv')

# Use on additional xeno-canto(CC-BY-NC-SA)
#sample_sub = pd.read_csv('/kaggle/input/birdclef-2023-additional/xeno-canto_audio_meta.csv')

# Use on additional xeno-canto(CC-BY-NC-ND)
#sample_sub = pd.read_csv('/kaggle/input/birdclef-2023-additional/xeno-canto_nd_audio_meta.csv')


# Use on training dataset(2023), no-call dataset, additional xeno-canto(CC-BY-NC-SA), additional xeno-canto(CC-BY-NC-ND)
# for c in sub.columns[1:]:
#     sample_sub[c] = 0

# Use on pretraining dataset
#for c in list(sample_sub.primary_label.unique()):
#    sample_sub[c] = 0


# sample_sub[competition_classes] = sample_sub[competition_classes].astype(np.float32)
# sample_sub

In [10]:
frame_limit_secs = None

all_logits_rst, filenames_rst = [], []

for index, sample_filename in enumerate(tqdm(test_samples)):
    all_logits_, filenames_ = predict_for_sample(sample_filename, sample_sub, index, frame_limit_secs=frame_limit_secs)
    all_logits_rst.append(all_logits_)
    filenames_rst.append(filenames_)

  0%|          | 0/24459 [00:00<?, ?it/s]

I0000 00:00:1716251388.450939      24 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [11]:
all_logits_rst = np.concatenate(all_logits_rst, axis=0)[:, 0, :]
filenames_rst = np.concatenate(filenames_rst, axis=0)

In [12]:
all_logits_rst.shape

(217738, 10932)

In [13]:
filenames_rst.shape

(217738,)

In [14]:
sample_sub = pd.DataFrame({"filename": filenames_rst})

In [15]:
sub = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
for c in sub.columns[1:]:
    sample_sub[c] = 0

/tmp/ipykernel_24/2565648264.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sample_sub[c] = 0
/tmp/ipykernel_24/2565648264.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sample_sub[c] = 0
/tmp/ipykernel_24/2565648264.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sample_sub[c] = 0
/tmp

In [16]:
columns_name = ['filename']

# Use on training dataset(2023), additional xeno-canto(and CC-BY-NC-SA), additional xeno-canto(and CC-BY-NC-ND)
for c in sub.columns[1:]:
    columns_name.append(c)
    
# Use on pretraining dataset
#for c in list(sample_sub.primary_label.unique()):
#    columns_name.append(c)
    
sample_sub = sample_sub[columns_name]
# sample_sub

In [17]:
def sigmoid(x):
    """Calculate sigmoid function."""
    return 1 / (1 + np.exp(-x))

for i in range(sample_sub.shape[0]):
    sample_sub.loc[i, competition_classes] = sigmoid(all_logits_rst[i, competition_class_map])

/tmp/ipykernel_24/269818503.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.884915997787175e-07' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sample_sub.loc[i, competition_classes] = sigmoid(all_logits_rst[i, competition_class_map])
/tmp/ipykernel_24/269818503.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.191766831558198e-06' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sample_sub.loc[i, competition_classes] = sigmoid(all_logits_rst[i, competition_class_map])
/tmp/ipykernel_24/269818503.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.987197712580382e-07' has dtype incompatible with int64, please explicitly cast to a compatible dtype fi

In [18]:
sample_sub

,filename,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,XC312771_5,3.884916e-07,5.191767e-06,4.987198e-07,5.526156e-10,3.456239e-06,1.735194e-07,7.046133e-07,3.334758e-09,6.938865e-06,...,8.030569e-08,4.614451e-06,3.378175e-05,4.646409e-06,4.366359e-06,0.010279,1.746979e-07,4.067421e-08,5.045283e-07,0.000040
1,XC312771_10,4.677325e-07,1.544179e-06,2.247659e-06,4.185980e-09,4.473692e-05,3.774830e-08,1.351667e-07,6.554360e-08,3.310415e-05,...,3.183021e-08,1.185043e-05,1.270174e-05,4.755683e-06,4.087824e-06,0.002651,4.620460e-08,1.365436e-07,1.948094e-06,0.000058
2,XC312771_15,7.170764e-07,3.206732e-08,9.434162e-07,1.320183e-08,3.615024e-05,3.315101e-08,6.028850e-08,7.534271e-07,1.783179e-06,...,3.451726e-07,9.004174e-07,1.018743e-05,5.320393e-06,5.897891e-06,0.001531,1.823510e-07,6.879516e-07,3.678197e-09,0.000005
3,XC37584_5,1.958438e-06,6.652931e-06,5.677631e-05,3.062173e-05,4.925393e-04,4.499684e-07,4.114494e-06,7.382372e-06,1.679205e-05,...,6.836347e-04,1.047248e-04,1.260674e-05,9.227016e-06,4.255813e-05,0.000040,1.637544e-05,1.113225e-05,1.621839e-04,0.000053
4,XC37584_10,1.855950e-05,2.990373e-05,8.296768e-05,1.215599e-05,5.216768e-05,2.941945e-07,3.227493e-05,3.208928e-06,1.767897e-05,...,2.290878e-04,2.055895e-06,6.358588e-05,9.143050e-05,1.266071e-04,0.000042,6.756880e-04,4.246928e-06,3.020777e-05,0.000069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217733,XC687073_35,1.168147e-05,2.610830e-06,2.918934e-07,7.888019e-09,2.157866e-05,2.563848e-07,5.731073e-08,5.469327e-08,4.349243e-07,...,7.325625e-08,3.972476e-07,4.660732e-05,1.449358e-07,1.919267e-05,0.000126,2.856008e-08,8.024902e-07,3.479819e-08,0.000023
217734,XC687073_40,5.717284e-06,2.874012e-05,2.628999e-06,5.056424e-06,2.019511e-05,1.195927e-05,3.084935e-07,7.898574e-07,5.869287e-08,...,1.359553e-07,5.829246e-06,4.671815e-04,1.427232e-06,2.924732e-05,0.007517,8.054351e-06,8.131275e-07,4.246755e-08,0.000173
217735,XC687073_45,2.215382e-06,7.052486e-08,1.212035e-08,2.575270e-09,9.826352e-07,1.967826e-07,1.801784e-08,1.708091e-07,2.477725e-08,...,3.472735e-07,2.953031e-08,3.838828e-07,5.415607e-07,3.609930e-07,0.000015,4.162464e-08,8.408087e-08,2.789992e-08,0.000037
217736,XC687073_50,6.778153e-06,9.944050e-07,2.368892e-06,1.049790e-07,8.303645e-05,1.052715e-06,5.271461e-07,3.418897e-06,7.837796e-06,...,4.402992e-07,2.512680e-06,3.712063e-05,4.301639e-06,4.453673e-05,0.000202,2.197330e-07,5.729516e-06,4.918342e-08,0.000045


In [19]:
sample_sub.to_csv("submission.csv", index=False)

In [20]:
sample_sub.head(48)

,filename,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,XC312771_5,3.884916e-07,5.191767e-06,4.987198e-07,5.526156e-10,0.000003,1.735194e-07,7.046133e-07,3.334758e-09,6.938865e-06,...,8.030569e-08,4.614451e-06,3.378175e-05,4.646409e-06,4.366359e-06,0.010279,1.746979e-07,4.067421e-08,5.045283e-07,0.000040
1,XC312771_10,4.677325e-07,1.544179e-06,2.247659e-06,4.185980e-09,0.000045,3.774830e-08,1.351667e-07,6.554360e-08,3.310415e-05,...,3.183021e-08,1.185043e-05,1.270174e-05,4.755683e-06,4.087824e-06,0.002651,4.620460e-08,1.365436e-07,1.948094e-06,0.000058
2,XC312771_15,7.170764e-07,3.206732e-08,9.434162e-07,1.320183e-08,0.000036,3.315101e-08,6.028850e-08,7.534271e-07,1.783179e-06,...,3.451726e-07,9.004174e-07,1.018743e-05,5.320393e-06,5.897891e-06,0.001531,1.823510e-07,6.879516e-07,3.678197e-09,0.000005
3,XC37584_5,1.958438e-06,6.652931e-06,5.677631e-05,3.062173e-05,0.000493,4.499684e-07,4.114494e-06,7.382372e-06,1.679205e-05,...,6.836347e-04,1.047248e-04,1.260674e-05,9.227016e-06,4.255813e-05,0.000040,1.637544e-05,1.113225e-05,1.621839e-04,0.000053
4,XC37584_10,1.855950e-05,2.990373e-05,8.296768e-05,1.215599e-05,0.000052,2.941945e-07,3.227493e-05,3.208928e-06,1.767897e-05,...,2.290878e-04,2.055895e-06,6.358588e-05,9.143050e-05,1.266071e-04,0.000042,6.756880e-04,4.246928e-06,3.020777e-05,0.000069
5,XC37584_15,2.208311e-05,1.121738e-04,5.189821e-06,3.954498e-06,0.000080,6.026964e-08,6.070746e-06,1.581473e-05,1.355157e-05,...,1.525890e-04,6.414959e-07,1.352564e-04,1.201911e-05,3.170825e-05,0.000196,2.513118e-05,8.328531e-06,1.118440e-04,0.000032
6,XC37584_20,3.444745e-06,1.338438e-05,1.778099e-05,2.925689e-04,0.000280,8.169286e-08,7.053259e-06,1.096259e-05,2.287204e-06,...,2.981306e-04,1.449832e-05,1.803310e-05,4.990886e-05,3.403559e-05,0.000296,7.854088e-05,3.396688e-05,1.315724e-05,0.000239
7,XC37584_25,7.818028e-06,3.560430e-05,2.128869e-06,5.743520e-07,0.000137,2.223316e-07,1.057540e-05,2.190992e-07,3.978147e-06,...,5.206589e-04,1.044577e-05,1.574588e-05,3.251001e-06,5.759125e-06,0.000022,3.133470e-06,1.619215e-05,2.291529e-05,0.000009
8,XC37584_30,2.769209e-05,3.013303e-05,5.346136e-07,6.125759e-07,0.000005,2.207975e-08,1.747839e-06,1.401017e-07,9.538298e-07,...,8.630793e-05,2.939824e-08,1.168006e-05,3.037609e-07,8.822352e-06,0.000423,1.993573e-05,8.725067e-07,3.351359e-07,0.000065
9,XC283501_5,7.295465e-05,1.897398e-06,4.894026e-07,1.643339e-07,0.000025,5.351860e-07,6.002469e-05,1.486682e-06,3.985058e-06,...,2.561114e-06,4.575315e-06,3.447736e-05,1.303484e-06,2.851009e-06,0.000264,1.566270e-05,2.100491e-06,3.672090e-05,0.000057
